In [4]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
%matplotlib inline  

## Data Load

In [31]:
def movieLensDataLoad(type):
## user 영화 별점 data 
    ratings = pd.read_csv(type+"/ratings.csv")

    ## movie meta(타이트,장르) data 
    movies = pd.read_csv(type+"/movies.csv")

    ## user가 영화에 tag를 기입한 data
    tags = pd.read_csv(type+"/tags.csv")
    # tags = pd.read_csv("/Users/goodvc/Documents/data-analytics/movie-recommendation/ml-20m/tags.csv")
    return ( ratings, movies, tags )

#ratings, movies, tags = movieLensDataLoad('ml-20m')

ratings, movies, tags = movieLensDataLoad('ml-latest-small/ml-latest-small')

In [32]:
print(ratings.shape)
print(movies.shape)
print(tags.shape)
ratings.head(3)

(100004, 4)
(9125, 3)
(1296, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


## User Based 별점 예측
* U x M vector Matrix를 만든다. ( key = userId / values = { movieID:rating }
* 이를 통해 나와 비슷한 user를 찾는다.

In [33]:
UM_matrix_ds = ratings.pivot(index='userId', columns='movieId', values='rating')
UM_matrix_ds.size

6083286

In [41]:
print(UM_matrix_ds.shape)
UM_matrix_ds.head(2)

(671, 9066)


movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
## 그럼 이제 최근접 이웃을 찾차 보자
## 
import math
from operator import itemgetter
from scipy.spatial import distance

def distance_cosine(a,b):
    return 1-distance.cosine(a,b)

def disance_corr(a,b):
    return 1-distance.correlation (a,b)

def distance_euclidean(a,b):
    return 1/(distance.euclidean(a,b)+1)

In [65]:
def nearest_neighbor_user( user, topN, simFunc):
    u1 = UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = {}
    
    ## Brote Force Compute
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid ==user:
            continue
        for i in ratedIndex:
            if False == math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        ## At least 3 intersection item
        if interSectionLen < 3:
            continue
            
        sim = simFunc(interSectionU1, interSectionU2)
        
        if math.isnan(sim) == False:
            nn[uid] = sim
            
        ## Top N returned
        return sorted(nn.items() , key=itemgetter(1))[: -(topN+1):-1]

In [71]:
%time nearest_neighbor_user(8, 20, distance_euclidean)

Wall time: 11 ms


[(2, 0.2449655295864104)]

In [73]:
def predictRating(userid, nn=100, simFunc=distance_euclidean):
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor ]

In [86]:
neightbor_movie =UM_matrix_ds.loc[[4,3]].dropna(1)

In [87]:
neightbor_movie

movieId,296,356,588,1197,1210,1580,2716
userId,,,,,,,
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,4.5,5.0,3.0,5.0,3.0,3.5,3.0
